<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Observability for AutoGen with Langfuse Integration" sidebarTitle: "AutoGen" logo: "/images/integrations/autogen_icon.svg" description: "Learn how to integrate Langfuse with AutoGen via OpenTelemetry using OpenLit for comprehensive tracing and debugging of your AI agent conversations." category: "Integrations" -->

# Integrate Langfuse with AutoGen

This notebook provides a step-by-step guide on integrating **Langfuse** with **AutoGen** to achieve comprehensive observability and debugging for your multi-agent LLM applications.

> **What is AutoGen?** [AutoGen](https://microsoft.github.io/autogen/stable/) ([GitHub](https://github.com/microsoft/autogen)) is an open-source framework developed by Microsoft for building LLM applications, including agents capable of complex reasoning and interactions. AutoGen simplifies the creation of conversational agents that can collaborate or compete to solve tasks.

> **What is Langfuse?** [Langfuse](https://langfuse.com) is an open-source LLM engineering platform. It offers tracing and monitoring capabilities for AI applications. Langfuse helps developers debug, analyze, and optimize their AI systems by providing detailed insights and integrating with a wide array of tools and frameworks through native integrations, OpenTelemetry, and dedicated SDKs.

## Getting Started

Let's walk through a practical example of using AutoGen and integrating it with Langfuse via OpenTelemetry for comprehensive tracing.

<!-- STEPS_START -->
### Step 1: Install Dependencies

<!-- CALLOUT_START type: "info" emoji: "⚠️" -->
> **Note:** This notebook utilizes the Langfuse OTel Python SDK v3. For users of Python SDK v2, please refer to our [legacy AutoGen integration guide](https://github.com/langfuse/langfuse-docs/blob/26a3aa9e935a0325b463e3fa585772fd20c23642/cookbook/integration_autogen.ipynb).
<!-- CALLOUT_END -->


In [ ]:
%pip install langfuse openlit "autogen-agentchat" "autogen-ext[openai]" -U

### Step 2: Configure Langfuse SDK

Next, set up your Langfuse API keys. You can get these keys by signing up for a free [Langfuse Cloud](https://cloud.langfuse.com/) account or by [self-hosting Langfuse](https://langfuse.com/self-hosting). These environment variables are essential for the Langfuse client to authenticate and send data to your Langfuse project.

In [ ]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your openai key
os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [3]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


### Step 3: Initialize OpenLit Instrumentation

Now, we initialize the [OpenLit](https://github.com/openlit/openlit) instrumentation. OpenLit automatically captures AutoGen operations and exports OpenTelemetry (OTel) spans to Langfuse.

In [ ]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True)

### Step 5: Basic AutoGen Application

Let's create a straightforward AutoGen application. In this example, we'll create a simple multi-agent conversation where an Assistant agent answers a user's question. This will serve as the foundation for demonstrating Langfuse tracing.

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4o")
agent = AssistantAgent("assistant", model_client=model_client)
print(await agent.run(task="Say 'Hello World!'"))
await model_client.close()

messages=[TextMessage(id='0935120c-2a27-4bb8-ad7f-eb5cb2b4902d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 8, 8, 32, 25, 422043, tzinfo=datetime.timezone.utc), content="Say 'Hello World!'", type='TextMessage'), TextMessage(id='2046975a-235c-4e46-be55-719bb79ced95', source='assistant', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=5), metadata={}, created_at=datetime.datetime(2025, 7, 8, 8, 32, 26, 265424, tzinfo=datetime.timezone.utc), content='Hello World! TERMINATE', type='TextMessage')] stop_reason=None


### Step 6: View Traces in Langfuse

After executing the application, navigate to your Langfuse Trace Table. You will find detailed traces of the application's execution, providing insights into the agent conversations, LLM calls, inputs, outputs, and performance metrics. The trace will show the complete flow from the initial user query through the agent interactions to the final response.

![Langfuse Trace](https://langfuse.com/images/cookbook/integration-autogen/autogen-example-trace.png)

You can also view the public trace here: [Langfuse Trace Example](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1b72c51fabed12ae7df83bfd4a09f545?timestamp=2025-06-06T11:31:33.965Z&display=details)
<!-- STEPS_END -->

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->